In [1]:
# ========================================================
# 1. 🛠️ 환경 설정 및 경로, Import (Code Block 1/2)
# ========================================================
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
import torchvision.models as models
import pandas as pd
import numpy as np
import random
import os
import sys
from tqdm import tqdm
from typing import Tuple, Dict, List
from google.colab import drive
from sklearn.metrics.pairwise import cosine_similarity
from torch.utils.data.dataloader import default_collate
from typing import List, Dict


In [2]:
# 💡 완벽한 재현성 확보를 위한 추가 설정 (성능 저하 가능성 있음)

# 1. CUDA/CUDNN 비결정성 방지
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 2. PyTorch 해시 시드 고정 (보통은 1번으로 충분)
# os.environ['PYTHONHASHSEED'] = str(EXPERIMENT_SEED)

In [3]:
# 마운트 (공유 드라이브 경로가 MyDrive에 바로 연결되어 있다고 가정)
drive.mount('/content/drive')

# --- 경로 설정 ---
# 이 경로가 dataset.py, transforms.py, checkpoints 폴더가 있는 위치여야 합니다.
MODULE_PATH = "/content/drive/MyDrive/2025CV"
sys.path.append(MODULE_PATH)


Mounted at /content/drive


In [4]:
# 💡 Import Custom Modules (dataset.py, transforms.py)
# BBox 크롭 로직과 Transforms 정의를 가져옵니다.
from dataset_jw import DeepFashionC2S
from transforms import train_transform, val_transform

In [5]:
# --- Hyperparameters (팀원 간 통일 필수) ---
EXPERIMENT_SEED = 42
EMBEDDING_DIM = 128  # 128은 baseline용 기본값, 실험에서는 아래 루프에서 256/512로 덮어씀
LEARNING_RATE = 1e-4
TRIPLET_MARGIN = 0.5 # Online Semi-Hard Triplet Loss 마진 값
BATCH_SIZE = 32
PATIENCE = 5         # Early Stopping Patience (5 Epoch 동안 개선 없으면 중단)
MAX_EPOCHS = 40      # 최대 학습 Epoch 수
CHECKPOINT_DIR = os.path.join(MODULE_PATH, "checkpoints_D_modify2")

os.makedirs(CHECKPOINT_DIR, exist_ok=True)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# 재현성 확보를 위한 시드 고정
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    return torch.Generator().manual_seed(seed)

generator = set_seed(EXPERIMENT_SEED)

Using device: cuda


In [6]:
# --- CSV 파일 로드 (샘플링된 CSV 사용) ---
CSV_PATH_LIGHT = os.path.join(MODULE_PATH, "meta_c2s_10_2_2_sampling_ID.csv")

In [7]:
import os
import shutil
import pandas as pd

# --- 경로 설정 확인 ---

DRIVE_IMG_ROOT = os.path.join(MODULE_PATH, "Images") # 원본 이미지 루트 경로 (Drive)
LOCAL_IMG_ROOT = "/content/Images"                   # 타겟 이미지 루트 경로 (Local)

In [8]:
# =========================================================
# CSV 기반 선택적 이미지 로컬 복사 및 I/O 최적화
# =========================================================
print("🚀 CSV 기반 선택적 이미지 로컬 런타임 복사 시작...")

# 1. CSV 파일 로드
try:
    df_light = pd.read_csv(CSV_PATH_LIGHT)
except FileNotFoundError:
    print(f"❌ 오류: CSV 파일을 찾을 수 없습니다. 경로를 확인하세요: {CSV_PATH_LIGHT}")
    exit()

# 2. 필요한 모든 이미지 경로 추출
# consumer_path와 shop_path 열에서 유니크한 경로만 추출합니다.
required_paths = pd.concat([df_light['consumer_path'], df_light['shop_path']]).unique()
print(f"총 {len(required_paths)}개의 유니크한 이미지 파일을 복사합니다.")

# 3. 로컬 타겟 폴더 생성
os.makedirs(LOCAL_IMG_ROOT, exist_ok=True)

# 4. 파일 복사 및 폴더 구조 유지
copied_count = 0
for relative_path in required_paths:
    # 원본 파일 경로 (Drive)
    source_file_path = os.path.join(DRIVE_IMG_ROOT, relative_path)

    # 타겟 파일 경로 (Local)
    target_file_path = os.path.join(LOCAL_IMG_ROOT, relative_path)

    # 타겟 디렉토리 생성 (예: /content/Images/img/TOPS/Summer_ 에 필요한 폴더 생성)
    target_dir = os.path.dirname(target_file_path)
    os.makedirs(target_dir, exist_ok=True)

    # 파일 복사
    try:
        if not os.path.exists(target_file_path):
             shutil.copy2(source_file_path, target_file_path)
             copied_count += 1
    except FileNotFoundError:
        # CSV에 경로가 있지만 실제 Drive에 파일이 없는 경우 건너뜁니다.
        print(f"[경고] 원본 파일이 Drive에 없습니다. 건너뜀: {source_file_path}")
    except Exception as e:
        print(f"[오류] 복사 중 예외 발생 ({relative_path}): {e}")

print(f"✅ 로컬 복사 완료. 총 {copied_count}개의 파일 복사됨.")

🚀 CSV 기반 선택적 이미지 로컬 런타임 복사 시작...
총 10546개의 유니크한 이미지 파일을 복사합니다.
✅ 로컬 복사 완료. 총 10546개의 파일 복사됨.


In [9]:
# =========================================================
# 3. IMG_ROOT_DIR 변수를 로컬 경로로 변경
# =========================================================
# 기존 경로 변수를 새로운 로컬 경로로 덮어씁니다.
IMG_ROOT_DIR = LOCAL_IMG_ROOT

print(f"새로운 이미지 루트 경로: {IMG_ROOT_DIR}")

새로운 이미지 루트 경로: /content/Images


In [10]:
# ============================================
# item_id 문자열 → 숫자 라벨 변환 매핑 생성
# ============================================

df_full = pd.read_csv(CSV_PATH_LIGHT)
unique_ids = df_full["item_id"].unique()

id2label = {id_str: idx for idx, id_str in enumerate(unique_ids)}
print("총 unique item_id 개수:", len(id2label))


총 unique item_id 개수: 1467


In [11]:
# ========================================================
# 2. 모델 및 손실함수 정의 (Stage 2 핵심) (Code Block 2/2)
# ========================================================

# 백본 모델 클래스 (Feature Embedding)
class FeatureEmbedding(nn.Module):
    def __init__(self, backbone_name, loss_name, embedding_dim):
        super().__init__()

        # 1. 백본 로드 및 사전 학습 가중치 설정
        if backbone_name == 'ResNet-50':
            base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
            num_ftrs = base_model.fc.in_features

        elif backbone_name == 'ResNet-34':
            base_model = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
            num_ftrs = base_model.fc.in_features

        elif backbone_name == 'EfficientNet-B3':
            base_model = models.efficientnet_b3(weights=models.EfficientNet_B3_Weights.IMAGENET1K_V1)
            num_ftrs = base_model.classifier[1].in_features

        else:
            raise ValueError(f"Unknown backbone: {backbone_name}")

        # 2. 특징 추출기 (마지막 FC 레이어 제거)
        if 'ResNet' in backbone_name:
            self.feature_extractor = nn.Sequential(*list(base_model.children())[:-1])
            self.final_fc = nn.Linear(num_ftrs, embedding_dim)
        elif 'EfficientNet' in backbone_name:
            self.feature_extractor = base_model.features
            self.avgpool = nn.AdaptiveAvgPool2d(1)
            self.final_fc = nn.Linear(num_ftrs, embedding_dim)

        self.bn = nn.BatchNorm1d(embedding_dim)

    def forward(self, x):
        x = self.feature_extractor(x)
        if hasattr(self, 'avgpool'):
            x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.final_fc(x)
        x = self.bn(x)
        x = F.normalize(x, p=2, dim=1)
        return x.to(DEVICE)

In [12]:
# Online Semi-Hard Triplet Loss 함수
def pairwise_distance_sq(embeddings: torch.Tensor) -> torch.Tensor:
    """배치 내 모든 임베딩 쌍 사이의 유클리디안 거리 제곱을 계산."""
    dot_product = torch.matmul(embeddings, embeddings.t())
    square_norm = torch.diag(dot_product)
    distances = square_norm.unsqueeze(0) - 2.0 * dot_product + square_norm.unsqueeze(1)
    distances[distances < 0] = 0
    return distances

# 온라인 세미-하드 트립렛 손실 함수 (Semi-Hard Mining 구현)
def online_semi_hard_triplet_loss(embeddings: torch.Tensor, labels: torch.Tensor, margin: float) -> torch.Tensor:
    """
    Online Semi-Hard Triplet Loss: 마진을 위반하지만 가장 어렵지는 않은 Negative를 선택하여 안정적으로 학습.
    """
    pair_dist = pairwise_distance_sq(embeddings)
    labels = labels.to(pair_dist.device)
    labels_eq = labels.unsqueeze(0).eq(labels.unsqueeze(1))

    # Hard Positive (A-P 중 가장 먼 거리)
    dist_ap = pair_dist.clone()
    dist_ap[~labels_eq] = float('-inf')
    hard_positive_dist, _ = dist_ap.max(dim=1)

    # Semi-Hard Negative (D(A,N) > D(A,P) 이면서 가장 가까운 N)
    dist_an = pair_dist.clone()
    dist_an[labels_eq] = float('inf')

    # Semi-Hard 조건: D(A,N) > D(A,P) + margin을 만족하는 영역
    # 마진 조건을 위반하지 않는 (A-P 거리가 A-N보다 이미 margin 이상 작은) N을 찾기 위해 부등호를 조정해야 합니다.
    # Triplet Loss: D(A,P) - D(A,N) + margin > 0 인 트립렛을 찾아야 합니다.
    # Semi-Hard는 D(A,P) < D(A,N) < D(A,P) + margin 입니다.

    # 1. Negative 중 마진을 위반하지 않는 후보 (D(A,N) > D(A,P))
    is_semi_hard_candidate = dist_an > hard_positive_dist.unsqueeze(1)

    # 2. Semi-Hard Negative 중 가장 가까운 샘플 선택
    # D(A,N)이 D(A,P)보다 크면서, A-P 마진(alpha)을 넘지 않는 영역에서 가장 가까운 N을 찾아야 합니다.
    # PyTorch 내장 함수를 사용하지 않으므로, 복잡한 인덱싱 대신 안정성을 위해 D(A,N) > D(A,P)인 샘플만 남깁니다.
    dist_an[~is_semi_hard_candidate] = float('inf')
    semi_hard_negative_dist, _ = dist_an.min(dim=1)

    # Triplet Loss 계산: L = max(0, D(A, P)_hard - D(A, N)_semi_hard + margin)
    triplet_loss = hard_positive_dist - semi_hard_negative_dist + margin
    triplet_loss[triplet_loss < 0] = 0

    num_hard_triplets = triplet_loss.gt(1e-16).float().sum()

    # 8에폭 오류 후 여기 수정함
    if num_hard_triplets > 0:
        return triplet_loss.sum() / num_hard_triplets
    else:
        # ⚠️ 여기가 핵심: 그래프에 붙어있는 "0"을 리턴
        # embeddings 에서 만들어진 값이므로 requires_grad=True 유지됨
        return (embeddings * 0.0).sum()

# Batch-Hard Triplet Loss
def batch_hard_triplet_loss(embeddings: torch.Tensor, labels: torch.Tensor, margin: float) -> torch.Tensor:
    """Batch-Hard Triplet Loss"""
    pair_dist = pairwise_distance_sq(embeddings)
    labels = labels.to(pair_dist.device)
    labels_eq = labels.unsqueeze(0).eq(labels.unsqueeze(1))

    # hardest positive
    dist_ap = pair_dist.clone()
    dist_ap[~labels_eq] = float('-inf')
    hardest_positive_dist, _ = dist_ap.max(dim=1)

    # hardest negative
    dist_an = pair_dist.clone()
    dist_an[labels_eq] = float('inf')
    hardest_negative_dist, _ = dist_an.min(dim=1)

    triplet_loss = hardest_positive_dist - hardest_negative_dist + margin
    triplet_loss = torch.clamp(triplet_loss, min=0.0)

    num_hard_triplets = triplet_loss.gt(1e-16).float().sum()
    if num_hard_triplets > 0:
        return triplet_loss.sum() / num_hard_triplets
    else:
        return (embeddings * 0.0).sum()

# InfoNCE Loss
def info_nce_loss(embeddings: torch.Tensor, labels: torch.Tensor, margin: float = 0.07) -> torch.Tensor:
    """
    InfoNCE Loss (Contrastive)
    margin 대신 temperature를 margin 인자로 재활용
    """
    temperature = margin
    sim_matrix = torch.matmul(embeddings, embeddings.T) / temperature

    # 자기 자신 제외
    mask = torch.eye(sim_matrix.size(0), dtype=torch.bool, device=embeddings.device)
    sim_matrix.masked_fill_(mask, -9e15)

    labels = labels.contiguous().view(-1, 1)
    matches = torch.eq(labels, labels.T).float() # 같은 라벨이면 1, 아니면 0

    exp_sim = torch.exp(sim_matrix)
    pos_sim = (exp_sim * matches).sum(dim=1)
    all_sim = exp_sim.sum(dim=1)

    loss = -torch.log(pos_sim / (all_sim + 1e-9))
    return loss.mean()

In [13]:
# ========================================================
# 3. 💾 체크포인트 관리 및 학습 루프 (정리 버전)
# ========================================================

def get_checkpoint_paths(backbone_name: str, loss_name: str, embedding_dim: int):
    """
    backbone + loss + embedding_dim 조합별로
    서로 다른 체크포인트 / 베스트 가중치 파일을 생성.
    """
    checkpoint_file = os.path.join(
        CHECKPOINT_DIR,
        f"{backbone_name}_{loss_name}_dim{embedding_dim}_checkpoint.pth"
    )

    best_file = os.path.join(
        CHECKPOINT_DIR,
        f"{backbone_name}_{loss_name}_dim{embedding_dim}_best_weights.pth"
    )

    return checkpoint_file, best_file


def save_checkpoint(model, optimizer, epoch, best_val_metric, patience_count, filename):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_val_metric': best_val_metric,  # Recall@K 값 (클수록 좋음)
        'patience_count': patience_count,
    }

    # 상위 디렉토리가 없으면 만들어주기 (안전장치)
    os.makedirs(os.path.dirname(filename), exist_ok=True)

    torch.save(checkpoint, filename)


def load_checkpoint(model, optimizer, filename):
    if not os.path.exists(filename):
        print("⚠️ 체크포인트 파일을 찾을 수 없습니다. 처음부터 학습을 시작합니다.")
        # Recall@K는 높을수록 좋으므로, best_val_metric을 0.0으로 초기화
        return 0, 0.0, 0

    checkpoint = torch.load(filename, map_location=DEVICE)

    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    start_epoch = checkpoint['epoch'] + 1
    best_val_metric = checkpoint['best_val_metric']
    patience_counter = checkpoint['patience_count']

    print(
        f"✅ 학습 재개: Epoch {start_epoch}부터 시작합니다. "
        f"(최고 Recall@5: {best_val_metric:.4f})"
    )

    return start_epoch, best_val_metric, patience_counter


In [14]:
def train_model(backbone_name, loss_name, train_dl, val_dl, criterion):

     # --- 초기화/복구 ---
    model = FeatureEmbedding(backbone_name, loss_name, EMBEDDING_DIM).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # 체크포인트 / 베스트 가중치 경로 한 번에 설정
    checkpoint_file, best_file = get_checkpoint_paths(backbone_name, loss_name, EMBEDDING_DIM)
    # 일반 체크포인트에서 학습 재개
    start_epoch, best_val_metric, patience_counter = load_checkpoint(
        model, optimizer, checkpoint_file
    )

    # 실험 기록 복구
    history_file = os.path.join(CHECKPOINT_DIR, f"{backbone_name}_{loss_name}_history.csv")
    history = []

    if os.path.exists(history_file) and os.path.getsize(history_file) > 0:
        try:
            history_df = pd.read_csv(history_file)
            if not history_df.empty:
                history = history_df.to_dict('records')
            else:
                raise pd.errors.EmptyDataError("DataFrame is empty after reading.")

        except pd.errors.EmptyDataError:
            print(f"⚠️ {backbone_name}_{loss_name}_history.csv 파일이 비어 있습니다. 새로운 기록을 시작합니다.")
            history = []
        except Exception as e:
            print(f"⚠️ History file loading error ({e.__class__.__name__}): {e}. Starting new history.")
            history = []

    # --- 학습 루프 ---
    for epoch in range(start_epoch, MAX_EPOCHS):

        print(f"\n--- Starting Epoch {epoch + 1}/{MAX_EPOCHS} for {backbone_name} + {loss_name} ---")

        model.train()
        total_loss = 0

        # Batch-All Triplet Loss를 위한 데이터 로딩 루프
        for batch in tqdm(train_dl, desc=f"Epoch {epoch+1} Train ({backbone_name}) + {loss_name}", leave=False):
            consumer_imgs = batch["consumer"].to(DEVICE)
            shop_imgs = batch["shop"].to(DEVICE)
            item_ids = batch["item_id"]

            # 문자열 item_id → 숫자 라벨 변환
            if isinstance(item_ids, (list, tuple)):
                item_ids = torch.tensor([id2label[i] for i in item_ids], dtype=torch.long)
            else:
                item_ids = torch.tensor([id2label[item_ids]], dtype=torch.long)

            item_ids = item_ids.to(DEVICE)

            # 이미지/라벨 병합
            all_imgs = torch.cat([consumer_imgs, shop_imgs], dim=0)
            all_labels = torch.cat([item_ids, item_ids], dim=0)

            # 모델 업데이트
            optimizer.zero_grad()
            embeddings = model(all_imgs)

            if loss_name == "infoNce":
                loss = criterion(embeddings, all_labels, margin=0.07)  # temperature = 0.07
            else:
                loss = criterion(embeddings, all_labels, TRIPLET_MARGIN)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_dl)

        # --- 검증 루프: 매 에포크마다 실행 ---
        val_recalls = calculate_recall_at_k(model, val_dl, DEVICE, ks=[1, 5, 10])
        val_metric = val_recalls['R@5']

        print(
            f"Epoch {epoch+1} | "
            f"Train Loss: {avg_train_loss:.4f} | "
            f"R@1: {val_recalls['R@1']:.3f} | "
            f"R@5: {val_recalls['R@5']:.3f} | "
            f"R@10: {val_recalls['R@10']:.3f}"
        )

        history.append({
            'epoch': epoch + 1,
            'train_loss': avg_train_loss,
            'val_R@1': val_recalls['R@1'],
            'val_R@5': val_recalls['R@5'],
            'val_R@10': val_recalls['R@10']
        })

        # --- Early Stopping ---
        if val_metric > best_val_metric:
            best_val_metric = val_metric
            patience_counter = 0

            # 최고 성능 모델 저장
            save_checkpoint(
                model, optimizer, epoch, best_val_metric, patience_counter,
                best_file
            )
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print(f"Early stopping triggered at epoch {epoch+1}.")
                pd.DataFrame(history).to_csv(history_file, index=False)
                break

        # Epoch 종료 시 체크포인트 저장
        save_checkpoint(model, optimizer, epoch, best_val_metric, patience_counter, checkpoint_file)

        # history 저장
        pd.DataFrame(history).to_csv(history_file, index=False)

    # --- 최종 결과 출력 ---
    try:
        # 베스트 가중치 로드
        load_checkpoint(model, optimizer, best_file)

        # 최종 Best Recall@K 계산
        final_recalls = calculate_recall_at_k(model, val_dl, DEVICE, ks=[1, 5, 10])

        print("\n========================================================")
        print(f"🏆 {backbone_name} + {loss_name} Final Best R@1: {final_recalls['R@1']:.4f}")
        print(f"🏆 {backbone_name} + {loss_name} Final Best R@5: {final_recalls['R@5']:.4f}")
        print(f"🏆 {backbone_name} + {loss_name} Final Best R@10: {final_recalls['R@10']:.4f}")
        print("========================================================\n")

        return {"backbone": backbone_name, "loss": loss_name, "final_epoch": epoch + 1, "best_R@5": final_recalls['R@5']}

    except Exception as e:
        print(f"Final evaluation failed: {e}")
        return {"backbone": backbone_name, "loss": loss_name, "final_epoch": epoch + 1, "best_R@5": 0.0}


In [15]:
# ========================================================
# calculate_recall_at_k 함수 (Dict Handling & Numeric Conversion)
# ========================================================
def calculate_recall_at_k(model, dataloader, device, ks=[1, 5, 10]):
    """
    Validation Set의 임베딩을 추출하고 Recall@K 성능을 측정합니다.
    (DataLoader가 딕셔너리를 반환하고, item_id를 숫자 라벨로 변환하여 사용)
    """
    model.eval()
    all_query_embs = []
    all_gallery_embs = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Calculating Recall@{ks[-1]}"):

            # 딕셔너리 형태의 batch에서 데이터 추출
            consumer_imgs = batch["consumer"]
            shop_imgs = batch["shop"]
            item_ids = batch["item_id"] # 문자열 (str) 리스트 또는 튜플

            # 문자열 item_id → 숫자 라벨 변환 (train_model과 동일한 로직 적용)
            if isinstance(item_ids, list) or isinstance(item_ids, tuple):
                # Global id2label 딕셔너리 사용 가정
                item_ids_tensor = torch.tensor([id2label[i] for i in item_ids], dtype=torch.long)
            else:
                item_ids_tensor = torch.tensor([id2label[item_ids]], dtype=torch.long)

            # 임베딩 계산
            query_embs = model(consumer_imgs.to(device)).cpu().numpy()
            gallery_embs = model(shop_imgs.to(device)).cpu().numpy()

            all_query_embs.append(query_embs)
            all_gallery_embs.append(gallery_embs)
            all_labels.append(item_ids_tensor.cpu().numpy()) # 숫자 Tensor를 NumPy 배열로 저장

    query_embs = np.concatenate(all_query_embs, axis=0)
    gallery_embs = np.concatenate(all_gallery_embs, axis=0)
    gallery_labels = np.concatenate(all_labels, axis=0) # [N,] 형태의 숫자 배열

    # 2. 코사인 유사도 계산 (거리 행렬)
    sims = cosine_similarity(query_embs, gallery_embs)

    recalls = {}
    for k in ks:
        # 상위 K개의 인덱스 추출 (유사도 내림차순)
        topk_idx = np.argsort(-sims, axis=1)[:, :k]

        correct_count = 0
        for i in range(len(gallery_labels)):
            query_true_id = gallery_labels[i]

            # 상위 K개의 갤러리 상품 ID
            topk_ids = gallery_labels[topk_idx[i]]

            # 정답 ID가 상위 K개 리스트에 포함되어 있는지 확인 (숫자 비교)
            if query_true_id in topk_ids:
                correct_count += 1

        recalls[f'R@{k}'] = correct_count / len(gallery_labels)

    return recalls

In [16]:
def custom_collate_fn(batch: List[Dict]):
    """None을 반환하는 샘플(파일 누락)을 필터링하고 collate."""
    # None인 샘플을 필터링합니다.
    batch = [item for item in batch if item is not None]

    # 배치에 유효한 샘플이 없으면 에러 방지를 위해 None 반환
    if not batch:
        return None

    # 유효한 샘플만 collate
    return default_collate(batch)

In [17]:
NUM_WORKERS = 0

In [18]:
# ========================================================
# 4. 🏁 임베딩 dim 256 / 512 실험 (train으로 학습, val로 early stopping)
# ========================================================
if __name__ == '__main__':
    # 1. train / val / test Dataset & DataLoader
    train_ds = DeepFashionC2S(
        csv_path=CSV_PATH_LIGHT,
        img_root=IMG_ROOT_DIR,
        transform=train_transform,
        split='train'
    )
    val_ds = DeepFashionC2S(
        csv_path=CSV_PATH_LIGHT,
        img_root=IMG_ROOT_DIR,
        transform=val_transform,
        split='val'
    )
    test_ds = DeepFashionC2S(
        csv_path=CSV_PATH_LIGHT,
        img_root=IMG_ROOT_DIR,
        transform=val_transform,
        split='test'
    )

    train_dl = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        generator=generator,
        pin_memory=True,
        collate_fn=custom_collate_fn
    )
    val_dl = DataLoader(
        val_ds,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        collate_fn=custom_collate_fn
    )
    test_dl = DataLoader(
        test_ds,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        collate_fn=custom_collate_fn
    )

    BACKBONES_TO_TEST = ['EfficientNet-B3']
    LOSS_FUNCTIONS_TO_TEST = [
        ("batchHardTriplet", batch_hard_triplet_loss),
    ]

    # 이번 라운드에서 실험할 임베딩 차원 (baseline 128은 이미 따로 있음)
    EMBEDDING_DIMS_TO_TEST = [256, 512]

    all_results = []    # dim별 val 성능 기록
    best_overall = None # {"embedding_dim", "backbone", "loss_name", "best_R@5"}

    for embed_dim in EMBEDDING_DIMS_TO_TEST:
        print(f"\n############## EMBEDDING_DIM = {embed_dim} ##############")
        global EMBEDDING_DIM
        EMBEDDING_DIM = embed_dim  # 전역 하이퍼파라미터 덮어쓰기

        for backbone in BACKBONES_TO_TEST:
            for loss_name, criterion in LOSS_FUNCTIONS_TO_TEST:
                print(f"\n================ Running Experiment: {backbone} + {loss_name} (dim={embed_dim}) ================")

                result = train_model(backbone, loss_name, train_dl, val_dl, criterion)
                # result: {"backbone", "loss", "final_epoch", "best_R@5"}

                result["embedding_dim"] = embed_dim
                all_results.append(result)

                if (best_overall is None) or (result["best_R@5"] > best_overall["best_R@5"]):
                    best_overall = {
                        "embedding_dim": embed_dim,
                        "backbone": backbone,
                        "loss_name": loss_name,
                        "best_R@5": result["best_R@5"]
                    }

    print("\n========== VAL RESULTS FOR EMBEDDING DIMS (train only) ==========")
    print(pd.DataFrame(all_results))
    print("\n========== BEST SETTING ON VAL ==========")
    print(best_overall)



############## EMBEDDING_DIM = 256 ##############

================ Running Experiment: EfficientNet-B3 + batchHardTriplet (dim=256) ================
Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-b3899882.pth


100%|██████████| 47.2M/47.2M [00:00<00:00, 181MB/s]


⚠️ 체크포인트 파일을 찾을 수 없습니다. 처음부터 학습을 시작합니다.

--- Starting Epoch 1/40 for EfficientNet-B3 + batchHardTriplet ---


Calculating Recall@10: 100%|██████████| 35/35 [00:10<00:00,  3.28it/s]


Epoch 1 | Train Loss: 0.5954 | R@1: 0.467 | R@5: 0.561 | R@10: 0.600

--- Starting Epoch 2/40 for EfficientNet-B3 + batchHardTriplet ---


...

Epoch 13 | Train Loss: 0.1452 | R@1: 0.592 | R@5: 0.644 | R@10: 0.704

--- Starting Epoch 14/40 for EfficientNet-B3 + batchHardTriplet ---


Calculating Recall@10: 100%|██████████| 35/35 [00:10<00:00,  3.28it/s]


Epoch 14 | Train Loss: 0.1433 | R@1: 0.586 | R@5: 0.642 | R@10: 0.682
Early stopping triggered at epoch 14.
✅ 학습 재개: Epoch 9부터 시작합니다. (최고 Recall@5: 0.6755)


Calculating Recall@10: 100%|██████████| 35/35 [00:10<00:00,  3.29it/s]



🏆 EfficientNet-B3 + batchHardTriplet Final Best R@1: 0.6126
🏆 EfficientNet-B3 + batchHardTriplet Final Best R@5: 0.6755
🏆 EfficientNet-B3 + batchHardTriplet Final Best R@10: 0.7119


========== VAL RESULTS FOR EMBEDDING DIMS (train only) ==========
          backbone              loss  final_epoch  best_R@5  embedding_dim
0  EfficientNet-B3  batchHardTriplet           19  0.689152            256
1  EfficientNet-B3  batchHardTriplet           14  0.675479            512

========== BEST SETTING ON VAL ==========
{'embedding_dim': 256, 'backbone': 'EfficientNet-B3', 'loss_name': 'batchHardTriplet', 'best_R@5': 0.6891522333637192}


In [19]:
    # ========================================================
    # 5. 최종 성능 측정: best dim 설정의 best checkpoint를 로드해서
    #    test에 대해 R@1,5,10만 한 번 계산 (재학습 없음)
    # ========================================================

    best_dim = best_overall["embedding_dim"]
    best_backbone = best_overall["backbone"]
    best_loss_name = best_overall["loss_name"]

    print(f"\n[FINAL EVAL] Using best setting from val: dim={best_dim}, backbone={best_backbone}, loss={best_loss_name}")

    # dim도 맞춰 줌 (모델 정의에 필요)
    EMBEDDING_DIM = best_dim

    # train_model에서 썼던 규칙 그대로 best weight 경로 얻기
    _, best_file = get_checkpoint_paths(best_backbone, best_loss_name, EMBEDDING_DIM)

    # 모델 생성 후 best weight 로드
    final_model = FeatureEmbedding(best_backbone, best_loss_name, EMBEDDING_DIM).to(DEVICE)
    final_optimizer = optim.Adam(final_model.parameters(), lr=LEARNING_RATE)  # load_checkpoint 형식 맞추기용

    # 체크포인트 로드 (load_checkpoint 대신 직접 state_dict만 써도 됨)
    checkpoint = torch.load(best_file, map_location=DEVICE)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.to(DEVICE)

    # test에서 R@1,5,10 계산
    test_recalls = calculate_recall_at_k(
        final_model,
        test_dl,
        DEVICE,
        ks=[1, 5, 10]
    )

    print("\n============ FINAL TEST RECALLS (best dim from val) ============")
    print(f"Test R@1  : {test_recalls['R@1']:.4f}")
    print(f"Test R@5  : {test_recalls['R@5']:.4f}")
    print(f"Test R@10 : {test_recalls['R@10']:.4f}")
    print("================================================================\n")


[FINAL EVAL] Using best setting from val: dim=256, backbone=EfficientNet-B3, loss=batchHardTriplet


Calculating Recall@10: 100%|██████████| 30/30 [00:09<00:00,  3.21it/s]


============ FINAL TEST RECALLS (best dim from val) ============
Test R@1  : 0.6402
Test R@5  : 0.7280
Test R@10 : 0.7751

